

## Побудова моделі лінійної регресії Prophet та її інтеграція у торговий термінал MT5


<div style="text-align: justify; text-indent: 30px;">
      Модель лінійної регресії Prophet була розроблена командою Facebook у 2017 році. Вона дозволяє прогнозувати зміни цін на фінансові інструменти з урахуванням сезонності, святкових днів та інших факторів. Самостійне використання модуля Prophet є вкрай неефективним, тому що навчання моделі слід проводити майже щоденно виходячи з того, що ціни на фінансові інструменти постійно змінюються. Відповідно постає питання щоденного отримання нових даних та перенавчання моделі. Більш того, дані щодо курсів фінанових інструментів можуть суттєво відрізнятися у кожного постачальника ліквідності. Отже, для створення прогнозів для реальних торгів бажано використовувати дані того постачальника ліквідності, який дає доступ до ринку. Цю проблему можливо вирішити, якщо інтегрувати блок розрахунку моделі лінійної реграсії Prophet у торговий термінал постачальника ліквідності. Для реалізації цього завдання був використаний торговий термінал MT5. 
</div>
<div style="text-align: justify; text-indent: 30px;">
    Торговий термінал MT5 – це програмне забезпечення для онлайн-торгівлі на фінансових ринках, розроблене компанією MetaQuotes Software. Він дозволяє трейдерам аналізувати ринкові дані, відкривати та закривати позиції, виконувати торгівельні операції та здійснювати фінансові транзакції. MT5 має ряд функцій, які роблять його популярним серед трейдерів, зокрема, великий набір інструментів аналізу ринку, можливість використовувати різні торгівельні стратегії та алгоритми, а також широкі можливості для роботизації торгівлі за допомогою експертів та радників (Expert Advisors) на основі мови програмування MQL5. За даними компанії MetaQuotes Software, розробника торгового терміналу MT5, станом на березень 2021 року, в світі налічувалося більше 400 000 зареєстрованих серверів торгівельних платформ MT5, на яких активно працюють мільйони трейдерів. MT5 є популярним торговим терміналом серед брокерських компаній, які надають своїм клієнтам доступ до онлайн-торгівлі на фінансових ринках, тому для реалізації завдання був обраний саме цей термінал.
</div>

   <div style="text-align: justify; text-indent: 30px;">
Для інтегрування моделі Prophet у термінал МТ5 потрібно розбити завдання на три етапи.
    
* Завантаження торгового радника на графік фінансвогого інструмента. Вибір типу моделі та параметрів майбутьньої моделі Prophet. Отримання даних від торгового терміналу та підготовка для розрахунків моделі.
    
* Побудова моделі Prophet у відповідності до заданих параметрів.
    
* Відображення отриманого передбачення на графіку торгівельного інструменту та при необхідності обрахування метрики передбачення.

Для виконання цих завдань перший та третій етапи потрібно виконати на мові програмування MQL5, а пункт 2 з побудови моделі – писати на мові Python. Торговий термінал МТ5 має можливість запускати скріпти написані на Python.
    </div>

### Вхідні параметри моделі

<div style="text-align: justify; text-indent: 30px;">
У процесі використання моделі Prophet виникає декілька питань, які тісно пов'язані з якістю отриманих результатів. Будь-які моделі лінійної регресії, засновані на часових рядах, дають гарні показники передбачення при відсутності провалів у даних, але ринок фінансових інструментів, зокрема, NYSE або  Forex  не працюють у вихідні дні та свята, що призводить до випадіння даних для прогнозу. Відповідно було прийняте рішення розглядати можливість застовування трьох моделей підготовки даних.</div>
 
 <div style="text-align: justify; text-indent: 30px;">
    1. Модель Ideal заснована на ідеї ущільнення даних в історії. В реальності модель для тренування завжди отримує дату початку розрахунку (Start) та кінцеву дату (End), після якої модель може проводити передбачення. Між цими датами завжди будуть знаходитися дні, в які не було торгів, а, отже, відсутні дані. Модель Ideal бере за основу кінцеву дату та від неї будує умовний календар у зворотньому напрямку, у якому відсутні вихідні та свята. Відповідно, кількість даних зменшується, ми отримуємо умовний календар для передбачення, який повністю співпадає із реальним календарем у кінцевій точці. Відповідно, передбачення, які робитиме модель, будуть розповсюджуватися на правильний період майбутнього. До недоліків цього методу можна віднести графік історії, який на відповідає реальним датам.</div>
 <div style="text-align: justify; text-indent: 30px;">    
    2.   Модель Normal передбачає передачу даних для обрахунку так як є. Із пропусками, але ці пропуски можливо компенсувати добудовою моделі свят. Коли кожний вихідний день передається моделі як свято з використанням механізму врахування свят, вбудованих у саму модель. Для цього потрібно додатково обрахувати окремий календар свят, який би повністю співпадав із вихідними днями. При цьому треба враховувати вплив свят на понеділок та п'ятницю. Модель  Prophet має таку можливість завдяки застосуванню налаштувань Holidays Wіndows.</div>
     <div style="text-align: justify; text-indent: 30px;">
    3. Модель Transform передбачає заповнення відсутніх даних середніми значеннями між даними п'ятниці та понеділка (можливо інших вихідних та святкових днів). Також для зручності та підвищення точності ці дні можуть додатково оголошуватися вихідними згідно з календарем для використання цих даних як додатковий регресійний фактор.</div>
 
<div style="text-align: justify; text-indent: 30px;"> 
Написана програма дає можливість обирати модель обробки даних і, відповідно, модель отриманого прогнозу.</div>
<div style="text-align: justify; text-indent: 30px;"> 
   Окремо від цього модель може враховувати окремі регресійні фактори. У якості регресійних факторів було прийняте рішення використовувати показник О-С. О-С — значення різниці між Open та Close денного бару. Ідея використання цього параметра заснована на наступній гіпотезі: якщо ця різниця значна та менше 0, то зростає ймовірність того, що наступного дня ціна фінансового інструмента збільшиться; якщо навпаки – різниця більше 0, то ціна з більшою ймовірністю зменшиться. Якщо різниця між Open та Close по модулю незначна у порівнянні з ATR (Avarage True Range) інструменту – це з більшою ймовірністю свідчить про  те, що інструмент буде знаходитися у стадії бокового тренду (flat).</div>
<div style="text-align: justify; text-indent: 30px;">   
   Модель може враховувати ще один регресійний фактор ATR (Avarage True Range). Він може свідчити про стадію, у якій знаходиться ринок.</div>
   
   
 <div style="text-align: center; "> <br> 
   ATR=|High-Low|</div> <br>
   
   
 <div style="text-align: justify; text-indent: 30px;">   
   Відповідно, при збільшенні ATR можна зробити висновок, що ринок знаходиться у періоді підвищеної волотильності, а, отже, прогноз має містити більшу амплітуду передбачення. При зниженні волатильності модель має видавати більш стриману амплітуду передбачуваних показників.</div>
  <div style="text-align: justify; text-indent: 30px;">  
   Програма дає можливість обрати, який регресійний фактор враховувати, або взагалі не використовувати жодних регресійних факторів.</div>
 <div style="text-align: justify; text-indent: 30px;">
   Ще одним не менш значущим параметром для моделі Prophet є параметр changepoint. Це ті точки, в яких проходить зміна поведінки функції. В моделі Prophet передбачена можливість автоматичного визначення кількості таких точок та встановлення коефіцієнтів впливу цього фактору на модель передбачення. Але в теорії трейдінгу існує поняття фракталів Вільямса, які визначають точки зміни поведінки тренду. Фрактал – це структура з п'яти барів, середній з яких є найбільшим (або найменшим) із цих барів. Така конструкція з великою  ймовірністю передбачає зміну тренду та є точкою changepoint. 
    <div style="text-align: justify; text-indent: 30px;"> 
    В програмі реалізована можливість двох варіантів врахування параметру changepoint:
        <ul>
            <li>Або  використовувати автоматичне  визначення точок (changepoint=auto) згідно з алгоритмом, вбудованим у модель</li> 
            <li>Або використати існуючі фрактали для виявлення точок (changepoint=Fractals_method). Для цього в програмі вбудовані функції, які збирають дані про фрактали на графіку, та окремий модуль, який проводить аналіз на істинність фракталів. На реальних графіках дуже часто зустрічаються випадки хибних фракталів, які не відповідають критеріям точок зміни тернду. Різницю між звичайними та хибними фракталами можна побачити на малюнку. Хибні фрактали виділені синім кольором.</li>
         </ul> </div>
   
![Фрактали Вільямса](GBPUSDDaily.png)
    

### Робота з програмою <br>


<div style="text-align: justify; text-indent: 30px;"> 
Програма, яка забезпечує збір та попередню обробку даних а також вивід результатів на дисплей  торгового терміналу, має назву Prophet_start та написана  на мові MQL5. Вона відноситься до категорії радників і знаходиться у компільованому вигляді у каталозі Experts, призначеному для розміщення радників. Для її запуску достатньо перетягнути  Prophet_start на робочу область графіку.</div>
<div style="text-align: justify; text-indent: 30px;">
Радник Prophet_start має певні обмеження.</div>
  <div style="text-align: justify; text-indent: 30px;">      
1. Програма розроблялася для застосування на ринку Forex, але вона може використовуватися для ринку акцій, comodities та деривативів, але іноді можуть виникати певні помилки, пов'язані з частими гепами інструментів.</div>
    <div style="text-align: justify; text-indent: 30px;">    
2. Програма призначена для роботи на денних графіках, а отже, програма робить щоденні  прогнози. У випадку спроби завантажити програму на графік, відмінний від таймфрейму D1, в журналі експерта буде виведено відповідне повідомлення та програма буде завершена.</div>
    <div style="text-align: justify; text-indent: 30px;">  
3. Історичний період, з яким може працювати програма, чітко не визначений. Остання дата, від якої може проводитися відлік, визначається наявністю історичних даних на сервері постачальника ліквідності і може відрізнятися у різних постачальників та на разних інструментах. Для цього в програмі передбачена можливість завантажити історію  опцією "Last history day" – тоді програма завантажить значення, які присутні в активному вікні терміналу. Слід зазначити, що часто термінал не показує всю історію, яка доступна для завантаження. Якщо вам потрібна більша кількість історичних данних, перед запуском програми необхідно спробувати завантажити додаткову історію з серверу постачальника послуг. Для цього слід перейти на початок графіку та спробувати підтягнути графік з історії. Можливо цю дію треба провести декілька разів. При активному завантаженні серверу  він може одразу не надавати доступ до історичних даних, тоді запит треба повторити або спробувати провести цю операцію у вихідний день, коли сервер не підтримує тікові операції. Після однократного завантаження повного списку історичних даних більше немає необхідності проводити цю операцію повторно. Історя буде зберігатися у файлах підкачки. Відповідно до цього, коли вперше запускається  Prophet_start, має сенс обирати параметр  "Last history day". В журналі радника ви побачите історичний період, що доступний раднику. Після цього ви будете приймати рішення, чи використовувати всю історію для передбачення, чи визначити свою дату, з якої буде здійснюватися розрахунок. </div>
    <div style="text-align: justify; text-indent: 30px;">    
4. Під час запуску радника одразу з'явиться вікно попередніх налаштувань. У вікні можна обрати 
дати початку та кінця періоду, на якому буде навчатися модель. За замовчуванням встановлено параметри "Last history day" для дати початку та "Сurrent_Day" для останнього дня періоду навчання моделі. У разі, якщо перший день історії буде обрано вручну та він буде передувати першому дню, який є в історії терміналу, програма автоматично змінить дату на "Last history day" та виведе відповідне попередження в журнал радника. Якщо дата кінця періоду, що введена вручну, буде знаходитися у майбутньому, програма автоматично проведе корекцію дати на  "Сurrent_Day", про що виведе відповідне попередження в журналі радника. Якщо дата кінця періоду  буде передувати даті  початку, програма видасть відповідне попередження та завершить роботу. У випадку, коли запрошені дати будуть випадати на святкові або вихідні дні, програма буде автоматично проводити корекцію дат, змінюючи їх на найближчі дати, які мають відповідні історичні дані. Про це програма буде виводити додаткові повідомлення в журнал радника та враховувати ці зміни при формуванні моделі та відображенні даних та результатів.</div>
<div style="text-align: justify; text-indent: 30px;">  
5. Кількість днів для створення прогнозу (Prediction_num) має бути цілим числом не більше 999.</div>
<div style="text-align: justify; text-indent: 30px;">  
6. Методи обробки даних, які використовує модель - Ideal, Normal, Transform, які були описані вище. </div>
<div style="text-align: justify; text-indent: 30px;">  
7. Метод визначення святкових днів (None, Country_Holidays, Weekends==Holidays).</div> 
    
    - None – святкові дні не враховуються    
    - Country_Holidays – святкові дні визначаються у відповідності до роботи американського ринку та конкретно Нью-Йоркської фодової біржі.     
     - Weekends==Holidays – будь-який вихідний день прирівнюється до свят і використовується як додатковий регресійний фактор або як додатковий фактор для формування моделі Transform та Normal.
     
<div style="text-align: justify; text-indent: 30px;">  
8. Метод визначення регресійних факторів(None, O-C, ATR): </div>
    
    - None – регресійний фактор не використовується
    - O-C – регресійний фактор Open-Close
    - ATR – регресійний фактор ATR. 
    
<div style="text-align: justify; text-indent: 30px;">  
9. Метод визначення точок changepoint (Auto, Fractals_method) використовується у відповідності із раніше наданим описом.</div>
<div style="text-align: justify; text-indent: 30px;">  
10. Можливі показники, по яких може бути проведено прогнозування (Open,High,Low,Close,ATR,MA). Слід зазначити, що модель обмежена одночасним прогнозуванням за трьома окремими незалежними показниками. Якщо будуть обрані однакові показники за двома чи більше пеараметрами, програма автоматично проведе корективи та видасть відповідне повідомлення. </div>
<div style="text-align: justify; text-indent: 30px;">  
11. При виборі пареметру МА (Moving avarage) останні дні історії, на яких не можливо вирахувати МА, будуть автоматично замінені даними першого дня, в якому з'являються історичні дані МА. При обчисленні МА використовується метод Linear Weighted / Weighted Close (HLCC/4). Період МА = 5 днів або повний торговий тиждень.</div>
<div style="text-align: justify; text-indent: 30px;">
12. Параметр ATR відрізняється при формуванні графіку від звичайних параметрів предикції. Тому результати обрахунку ATR будуть виводитися в окремому вікні стандартного індикатора ATR. Порівняння реальних даних та предикції буде здійснюватися окремо. Програма самостійно та автоматично проведе виклик необхідних вікон.</div>
<div style="text-align: justify; text-indent: 30px;">
13. Всі ці показники можуть бути обрані вручну, або є можливість створення необмеженої кількості файлів налаштувань з різними обраними комбінаціями параметрів та безпосереднього завантаження файлу налаштування.</div>

Зовнішній вигляд вікна ввода даних має такий вигляд ![Вікно ввода](Model.jpg)



<div style="text-align: justify; text-indent: 30px;">
<br>    
Після обрання всіх необхідних параметрів програма автоматично запуститься, про що буде свідчити відповідний значок у правому верхньому куті екрану, та  створить файл в робочому каталозі Files із вихідними даними для моделі Prophet.</div>

### Робота моделі Prophet

<div style="text-align: justify; text-indent: 30px;">
Після того, як перша частина коду програми відпрацювала та підготовула необхідні дані у відповідності до потреб моделі Prophet, в журналі радника вийде відповідне повідомлення та пропозиція запустити скрипт, написаний на Python. Після цього з каталогу Scripts потрібно запустити скрипт Prophet_model.  Для запуска відповідного скрипта достатньо перетягнути його в активне вікно терміналу.</div>
  <div style="text-align: justify; text-indent: 30px;">  
Найбільш важливі перетворення даних та розрахунки проводяться в модулі Prophet_model. На жаль, на сьогодні немає можливості об'єднати два модуля в один файл. Термінал МТ5 виконаний з додатковими вимогами до безпеки та не дозволяє стороннім програмам видавати команди терміналу. Більш того, термінал працює з файлами, які знаходяться виключно у "пісочниці" терміналу та мають обмежений доступ.</div>
<div style="text-align: justify; text-indent: 30px;">  
Важливо відмітити, що у вимогах моделі Prophet є обов'язкова умова про те, що регресійний фактор та святкові дні мають бути завантажені на період майбутнього прогнозування. Відповідно до цього в програмі Prophet_model спочатку проводиться передбачення  регресійного фактору із застосуванням автоматичних параметрів, а потім регресійний фактор додається до моделі на етапі навчання та передбачення.</div>
    <div style="text-align: justify; text-indent: 30px;">  
Виконання обчислень займає певний час та залежить від періоду, заданого для обчислення, моделі та параметрів. В результаті роботи скрипту  Prophet_model створюються декілька файлів у залежності від кількості параметрів, за якими ми запросили створити прогноз. В цей час програма Prophet_Start буде очікувати результати роботи модуля Prophet_model та кожну  секунду проводити опитування, чи завершила  Prophet_model свою роботу. Після завершення роботи Prophet_model самостійно вивантажується з пам'яті комп'ютера та не потребує додаткових дій від користувача. Далі Prophet_Start починає процес побудови графічних результатів отриманих прогнозів.</div>


### Вивід результатів розрахунків

 <div style="text-align: justify; text-indent: 30px;">  
Программа Prophet_model самостійно автоматично виводить отриману інформацію на екран основного графіку терміналу. Для зручності користувача графік предикції змінює колір безпосередньо на періоді предикції. Тут користувач має можливість оцінити логістичну криву, побудовану моделлю, та порівняти її з реальним графіком. У випадку, якщо період предикції знаходиться в історії графіку, тобто ми запускаємо модель для підбору параметрів для навчання, тоді до графіків на екран буде додатково виводитися параметр MAE (Mean Absolut Error) по кожному параметру окремо за період передбачення. Для зручності реалізована можливість переміщення цієї мітки в рамках екрану терміналу. Також слід зазначити, що ми використовуємо адаптовану МАЕ. Вона буде показувати абсолютне відхилення у пунктах торгової операції. Наприклад, якщо валютна пара котується із точністю 5 знаків після коми, то МАЕ автоматично помножиться на 100000 та надасть інформацію з відхилення у пунктах котування. Аналогічно буде проводитися перетворення для інструментів із іншою кількістю знаків після коми. Для цього модель автоматично отримує властивості валюти котування та відповідно проводить коригування МАЕ. Також до важливих аспектів роботи програми слід віднести наступне: якщо період предикції випадає на вихідні дні, програма автоматично переносить ці дати на робочі дні – тим самим модель адаптується до вимог торгового терміналу. Отже, іноді користувач, запросивши предикцію на 7 днів, отримає реальні результати за 9 днів. Хоча фактично просто дні вихідних будуть вийняті з періоду. Після завершення роботи користувач має можливість завершити виконання радника. Для цього він повинен у стандартний спосіб зняти програму радника з виконання. У цьому випадку програма перед завершенням роботи прибере графіки, які вона побудувала, та видалить всі файли з данними, які були створені для роботи моделі, для того, щоб не перевантажувати комп'ютер зайвими файлами.</div>

![Predict](prediction.png)

### Фіксація результатів роботи

 <div style="text-align: justify; text-indent: 30px;"> 
Виходячи з того, що модель може використовуватися на великій кількості інструментів та її навчання та підбір гіперпараметрів вимагає проведення певної кількості обрахунів, у програмі реалізована обов'язкова функція ведення журналу. Тобто при виконанні кожного обрахунку всі записи параметрів та результати, якщо такі були, додаються у файл prophet_journal.csv. Цей журнал створює додатковий датафрейм, який буде використовуватися для подальшого аналізу користоувачем. </div>

Обов'язкові записи в журналі :

   * фінансовий інструмент
   * дата початку обрахунку
   * дата кінця обрахунку
   * період передбачення
   * MAE (якщо це не передбачення)
   * параметр за яким проводилася предикція
   * тип моделі
   * holidays модель свят
   * регресійний фактор
   * модель changepoint
   * growth : тип моделі зростання. Може бути "linear" або "logistic".
   * n_changepoint s: кількість точок зміни тренду.
   * changepoint_prior_scale :   параметр, що визначає гладкість змін тренду.
   * seasonality_mode :   тип сезонності. Може бути "additive" або "multiplicative".
   * seasonality_prior_scale :   параметр, що визначає гладкість сезонності.
   * holiday_prior_scale :   параметр, що визначає вплив свят на прогноз.
   * seasonality :   список сезонних компонентів моделі.
   * yearly_seasonality :   параметр, що визначає щорічну сезонність.
   * weekly_seasonality :   параметр, що визначає щотижневу сезонність.
   * daily_seasonality :   параметр, що визначає щоденну сезонність.
   * month_seasonality :   параметр, що визначає місячну сезонність.
   * uncertainty_samples :   кількість зразків для оцінки невизначеності.
   * interval_width :   ширина інтервалу прогнозування.
   * mcmc_samples :   кількість зразків у алгоритмі Монте-Карло.
   * stan_backend :   тип бекенду для алгоритму Монте-Карло.
    
    
 <div style="text-align: justify; text-indent: 30px;">
Сам журнал має особливу цінність для аналізу МАЕ. Якщо ми сьогодні робимо передбачення та не маємо можливість обрахувати МАЕ, то після проходження періоду предикції завдяки журналу ми можемо повернутися та провести відповідний розрахунок із тими самими параметрами та дізнатися, наскільки прогноз фактично відрізнявся від майбутнього факту.</div>


###  Попередній аналіз роботи моделі

 <div style="text-align: justify; text-indent: 30px;">
    Для проведення аналізу роботи моделі проводилась певна кількість числових експериментів. Частково результати цих обрахунків можливо подивитися в журналі. 
</div> 

In [1]:
import pandas as pd
df=pd.read_csv("result_test.csv", delimiter=";")
df

,Error,Start,End,Mark,Metod,Predict_num,Changepoint_range,Changepoint_prior_scale,yearly_seasonality,weekly_seasonality,...,n_changepoints,changepoints,seasonality_mode,seasonality_prior_scale,mcmc_samples,interval_width,uncertainty_samples,stan_backend,holidays,holidays_prior_scale
0,0.022082,2019-01-01,2023-03-15,high,ideal_absolut,3,0.990,0.30,auto,1,...,25,None,additive,10,0,0.08,1000,None,None,10
1,0.021369,2004-03-12,2023-03-23,high,ideal_absolut,3,0.990,0.30,auto,1,...,25,None,additive,10,0,0.08,1000,None,None,10
2,0.048620,2005-01-01,2023-03-23,high,ideal_absolut,3,0.990,0.30,auto,1,...,25,None,additive,10,0,0.08,1000,None,None,10
3,0.045277,2006-01-01,2023-03-23,high,ideal_absolut,3,0.990,0.30,auto,1,...,25,None,additive,10,0,0.08,1000,None,None,10
4,0.041824,2007-01-01,2023-03-23,high,ideal_absolut,3,0.990,0.30,auto,1,...,25,None,additive,10,0,0.08,1000,None,None,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,0.003584,2013-01-01,2017-03-23,atr,ideal_absolut,3,0.999,0.55,5,5,...,250,None,additive,100,0,0.01,1000,None,None,10
165,0.008707,2012-01-01,2016-03-23,atr,ideal_absolut,3,0.999,0.55,5,5,...,250,None,additive,100,0,0.01,1000,None,None,10
166,0.005890,2011-01-01,2015-03-23,atr,ideal_absolut,3,0.999,0.55,5,5,...,250,None,additive,100,0,0.01,1000,None,None,10
167,0.004125,2010-01-01,2014-03-23,atr,ideal_absolut,3,0.999,0.55,5,5,...,250,None,additive,100,0,0.01,1000,None,None,10


<div style="text-align: justify; text-indent: 30px;"> 
Слід зазначити, що дуже неочікувані результати розрахунків показує модель Normal. Модель Normal застосовує підготовку даних із пропусками вихідних днів. У процесі обрахунків модель чітко вираховує у періоді передбачення вихідні дні але у ці дні вона видає різкі сплески відхилення даних від прогнозованих, що гарно видно на наступному малюнку.</div> 
 

![Normal](Normal_eraption.jpg)

В цілому модель показує відносно непогані результати на всьому періоді формування логістичної кривої.

![Normal](GBPCADDaily.png)

### Висновки

<div style="text-align: justify; text-indent: 30px;"> 
Створені програми Prophet_Start та Prophet_model дозволяють зручно та швидко аналізувати ринок фінансових інструментів та робити прогнози його розвитку. Але найбільшу цінність становить саме середовище, яке дозволяє проводити обрахунки та підбирати параметри для моделі.</div>
<div style="text-align: justify; text-indent: 30px;"> 
Для кожного окремого фінансового інстументу знадобиться проводити конкретну підгонку моделі, включаючи не тільки підбір гіперпараметрів, а також і період навчання моделі. Попередні обрахунки показали, що найбільш важливим фактором для роботи моделі виступають не додаткові регресійні фактори, а періоди, які використовуються для навчання. Відповідно до цього, алгоритм підгонки моделі слід починати з підбору періоду навчання, а потім переходити до налаштування гіперпараметрів. Обрахунки на парі GBP/CAD показали, що оптимального результату можливо добитися з переіодом навчання у 5-6 років. Що більшу історію брати, то гірші показники ми отримуємо.</div>
    <div style="text-align: justify; text-indent: 30px;"> 
    Найбільш оптимальною є наступна стратегія підгонки моделі:</div>
    <div style="text-align: justify; text-indent: 30px;">
    1. Проводити предикцію за 3 дні до сьогоднішньої дати та намагатися отримати найоптимальніші параметри MAE за ці дні.</div>
    <div style="text-align: justify; text-indent: 30px;">
    2. Після цього проводити прогноз сьогоднішнього дня із періодом предикції на 2-3 дні. </div>
    <div style="text-align: justify; text-indent: 30px;">
    Для кожного фінансового інструмента методики підбору параметрів та періоду навчання будуть відрізнятися.</div>
 <div style="text-align: justify; text-indent: 30px;"> </div>
На сьогодні вважаю, що використання моделі для реальних торгів є дуже ризиковим, але вона показує непогані результати у якості абсолютно нового торгового індикатора і буде представляти цінність для трейдерів, які віддають перевагу індикаторному стилю торгівлі. При її спільному використанні з іншими індикаторами можна отримати декілька нових торгових стратегій.</div>


### Додатки

#### Код програми Prophet_Start


In [ ]:
//+------------------------------------------------------------------+
//|                                                        Test3.mq5 |
//|                                  Copyright 2023, MetaQuotes Ltd. |
//|                                             https://www.mql5.com |
//+------------------------------------------------------------------+

#property copyright "Copyright 2023"
#property link      "https://facebook.github.io/prophet/"
#property version   "1.00"
#property description "Model-Prophet used Facebook Prophet metod for building chart of prediction price instrument. Author - Sharovarov Aleksandr sharovarovaleksandr@gmail.com"

// Формуємо заголовок для введення даних та визначення методу розрахунку
#property script_show_inputs
enum marks
  {
   sd0=0,  // None
   sd1=1,  // Open
   sd2=2,  // High
   sd3=3,  // Low
   sd4=4,  // Close
   sd5=5,  // MA
   sd6=6,  // ATR
  };
enum metod
  {
   md0=0,   // Ideal
   md1=1,   // Normal
   md2=2,   // Transform
  };
enum regres
  {
   rg0=0,   // None
   rg1=1,   // Open-Close
   rg2=2,   // ATR
  };
enum chang
  {
   ch0=0,   // Auto
   ch1=1,   // Fractals_method
  };
enum holi
  {
   hd0=0,   // None
   hd1=1,   // Country_Holidays
   hd2=2,   // Weekends=Holidays
  };
// Ввод констант заголовку
input string   Start_Date_YYYY─MM─DD = "Last_History_Day";
input string   End_Date_YYYY─MM─DD = "2023-04-20";
input int      Pridiction_Period = 5;
input metod    Metod_Processing_Data = md2;
input regres   Regression_Metod=rg0;
input chang    Changepoint_Metod=ch0;
input holi     Holiday_Metod=hd0;
input marks    Prediction_Mark_1 = sd2;
input marks    Prediction_Mark_2 = sd3;
input marks    Prediction_Mark_3 = sd0;


//Блок оголошення змінних
datetime Start, End, tim, testdata, testdata_new, load_time[20000]; //Виходимо з гіпотези що кількість барів в історії не перевищує 20000 (75 років)
bool Calc_MAE, Change_Prophet, Today, files_ready, graph_ready;
string filename, name, first_str, next_str, Start_Date_YYYY─MM─DD1,End_Date_YYYY─MM─DD1, File_forecast_name, mk[3], rab_str, str_result[5];
int i, j, k, l, num_bar, num_mk, m, n1, fil, IMA1, IFRAC1, IATR1, bar_shift, result_shift, ima_day, last_bar;
double open, high, low, close, up_fractal[], down_fractal[], ma[], atr[], regres_oc, regers_atr, load_data[3][3][20000], MAE, XMAE;
string mark_array[6]= {"open","high","low","close","ma","atr"};
string obj_name, name_graph[3], obj_name_ma;

MqlDateTime strt, ennd, loc_time, bar_time;
//+------------------------------------------------------------------+
//| Expert initialization function                                   |
//+------------------------------------------------------------------+
int OnInit()
  {
   Print("Prophet starts to work. Your Starts day - ", Start_Date_YYYY─MM─DD,"  End day - ",End_Date_YYYY─MM─DD);

   Change_Prophet=false; // Змінна буде визначати, чи потрібна корекція графіка, якщо End_Day=Weekend, Holiday
   Today=false;          // Змінна визначає день, в який проходять розрахунки
   files_ready=true;     // Змінна, яка перевіряє, чи існують файли з даними
   graph_ready=true;     // Змінна, яка перевіряє, чи побудовані вже графіки
   TimeLocal(loc_time);  // Зчитуємо час сервера
   Calc_MAE=true;        // Змінна буде визначати, це тестовий розрахунок чи реальний прогноз (розраховувати MAE чи ні)
   name = Symbol();      // Визначаемо символ, на якому будемо проводити розрахунок

// Перевіряємо відповідність графіків таймфрейму
   if(PERIOD_D1 != Period())
     {
      Print("Script works only D1 timeframe. Please try again!");
      PlaySound("alert2");
      ExpertRemove();
     }

// Перетворюємо дати у формат datetime та аналізуємо їх на можливі помилки
   if(Start_Date_YYYY─MM─DD == "Last_History_Day")
     {
      Start=iTime(name,PERIOD_CURRENT,Bars(Symbol(),PERIOD_CURRENT)-1);
     }
   else
     {
      Start=StringToTime(Start_Date_YYYY─MM─DD);
     }
   if(End_Date_YYYY─MM─DD == "Current_Day")
     {
      End=TimeCurrent();
      Calc_MAE=false;
      Today=true;
     }
   else
     {
      End=StringToTime(End_Date_YYYY─MM─DD);
      if(End == iTime(name,PERIOD_CURRENT,0))
        {
         End=TimeCurrent();
         Calc_MAE=false;
         Today=true;
        }
     }

// Перевірка введених дат на послідовніть
   if(Start >= End)
     {
      Print("Error rise the End_Date<=Start_Date");
      PlaySound("alert2");
      ExpertRemove();
     }

// Перевірка Start_Date на відповідність історії
   if(Start < iTime(name,PERIOD_CURRENT,Bars(Symbol(),PERIOD_CURRENT)-1))
     {
      Start=iTime(name,PERIOD_CURRENT,Bars(Symbol(),PERIOD_CURRENT)-1);
      Print(" The Start_Date is less then last history day and will be changed at ", Start);
     }
// Перевірка та корегування Start_Date на відповідність робочим дням
   n1=Bars(Symbol(),PERIOD_CURRENT,Start,TimeCurrent())-1;
   testdata=iTime(Symbol(),PERIOD_CURRENT,n1);
   if(Start != testdata)
     {
      Start=testdata;
      Print("Your Start_Date falls on a Weekend or Holiday and will be changed to ",Start);
     }

// Перевірка та корегування End_Date на відповідність останньому дню історії графіка
   if(End > TimeCurrent())
     {
      End=TimeCurrent();
      Calc_MAE=false;
      Today=true;
      Print("Your End_Date is later or equal than today's date end will be changed at ", End);
     }

// 6 та 0 - службові константи структури day_of_week - Субота та Неділя відповідно
   if(loc_time.day_of_week == 6 || loc_time.day_of_week == 0 || (loc_time.day_of_week == 1 && Today))
     {
      result_shift=2;
      Change_Prophet=true;
     }

// Перевірка, чи випадає End_Date на вихідні, та корекція дати
   if(!Today)
     {
      n1=Bars(Symbol(),PERIOD_D1,End,TimeCurrent())-1;
      testdata=iTime(Symbol(),PERIOD_D1,n1);
      TimeToStruct(testdata,strt);
      if(strt.day_of_week==5)
        {
         result_shift=2;
         Change_Prophet=true;
        }
      if(End != testdata)
        {
         Change_Prophet=true;
         n1=Bars(Symbol(),PERIOD_D1,End,TimeCurrent());
         testdata_new=iTime(Symbol(),PERIOD_D1,n1);
         TimeToStruct(testdata_new,ennd);
         result_shift=strt.day-ennd.day-1;
         End=testdata_new;
         Print("Your End_Date falls on a Weekend or Holiday and will be changed to ",End);
         Print("Don't worry, we remember this and will fix the results!");
        }
     }

// Визначаємо необхідність розрахунку MAE
   if(Bars(Symbol(),PERIOD_D1,End,TimeCurrent())-1<Pridiction_Period)
     {
      Calc_MAE=false;
     }

// Визначаємо наявну кількість барів в історії
   num_bar = Bars(Symbol(),PERIOD_CURRENT,Start,End)-1;

// Визначаємо номер бара, який припадає на точку Start
   bar_shift=iBarShift(Symbol(),PERIOD_CURRENT,Start);


// Відкриваємо файл у форматі csv для формування вихідних даних
   filename="Prop"+name+TimeToString(TimeCurrent(),TIME_DATE)+".csv";
   fil=FileOpen(filename, FILE_WRITE|FILE_CSV|FILE_ANSI);

// Перевіряємо, чи визначені параметри для роботи радника
   if(Prediction_Mark_1 == sd0 && Prediction_Mark_2 == sd0 && Prediction_Mark_3 == sd0)
     {
      Print("No <Prediction_Mark> parameter  defined for the script operation. Please try again!");
      PlaySound("alert2");
      ExpertRemove();
     }

// Перевіряємо, чи не задвоюються параметри
   if(((Prediction_Mark_1 == Prediction_Mark_2) && Prediction_Mark_1 != sd0) ||
      ((Prediction_Mark_1 == Prediction_Mark_3) && Prediction_Mark_1 != sd0) ||
      ((Prediction_Mark_2 == Prediction_Mark_3) && Prediction_Mark_2 != sd0))
     {
      Print("Parameters <Prediction_Mark> are match. Please try again!");
      PlaySound("alert2");
      ExpertRemove();
     }

// Перевіримо необхідність створення хендлів для індикаторів та завантажимо необхідні дані індикатору МА
   if(Prediction_Mark_1 == sd5  || Prediction_Mark_2 == sd5 ||Prediction_Mark_3 == sd5)
     {
      ima_day=5;  // змінна визначає період створення МА прийнята з розрахунку 5-денної робочої неділі
      IMA1=iMA(name,PERIOD_CURRENT,ima_day,0,MODE_LWMA,PRICE_WEIGHTED);
      ArraySetAsSeries(ma,false); // ma массив для зберігання значень MA
      if(CopyBuffer(IMA1,0,Start,End,ma) < 0)
        {
         Print("CopyBuffer IMA1 error =",GetLastError());
        }

      //  Якщо ми запросили МА за період, який передує Start date, то перші  ima_day елементів масиву ma будуть заповнені числами 1.0Е+378
      //  У такому випадку видати попередження, запросити переініціалізувати радника з правильною  Start date
      for(i=0; i<=ima_day; i++)
        {
         if(ma[i]>100000000)  // виходимо із гіпотези, що реально не існує інструменту з ціною більше 100000000$
           {
            Print("We use the MA=",ima_day,"(day) parameter.");
            Print("The Start date must differ by at least ",ima_day," days from the last date in the history.");
            Print("Please change the Start date and try again.");
            PlaySound("alert2");
            ExpertRemove();
           }
        }

     }

// Перевіримо необхідність завантаження та завантажимо значення  up_fracta, down_fractal з графіка
   if(Changepoint_Metod == ch1)
     {
      IFRAC1=iFractals(name,PERIOD_CURRENT);
      ArraySetAsSeries(up_fractal,false);
      ArraySetAsSeries(down_fractal,false);
      if(CopyBuffer(IFRAC1,0,Start,End,up_fractal) < 0)
        {
         Print("CopyBuffer IFRAC1 error =",GetLastError());
        }
      if(CopyBuffer(IFRAC1,1,Start,End,down_fractal) < 0)
        {
         Print("CopyBuffer IFRAC2 error =",GetLastError());
        }
     }

// Перевіримо необхідність створення хендлів для індикаторів та завантажимо необхідні дані індикатору ATR
   if(Regression_Metod == rg2 || Prediction_Mark_1 == sd6 || Prediction_Mark_2 == sd6|| Prediction_Mark_3 == sd6)
     {
      IATR1=iATR(name,PERIOD_CURRENT,1);
      ArraySetAsSeries(atr,false);
      if(CopyBuffer(IATR1,0,Start,End,atr) < 0)
        {
         Print("CopyBuffer ATR1 error =",GetLastError());
        }
     }

// Формуємо заголовок датафрейму у якості строки first_str, для цього додаємо до строки ті колонки, які увійдуть до вихідного файлу даних
// Колонки майбутнього датафрейму будуть мати назви відповідно до доданих текстових значень
   first_str="ds;";
   switch(Prediction_Mark_1)
     {
      case sd0:
         break;
      case sd1:
         first_str=first_str+mark_array[0]+";";
         break;
      case sd2:
         first_str=first_str+mark_array[1]+";";
         break;
      case sd3:
         first_str=first_str+mark_array[2]+";";
         break;
      case sd4:
         first_str=first_str+mark_array[3]+";";
         break;
      case sd5:
         first_str=first_str+mark_array[4]+"="+IntegerToString(ima_day)+";";
         break; // До назви колонки МА додається період створення ima_day
      case sd6:
         first_str=first_str+mark_array[5]+";";
         break;
     }

   switch(Prediction_Mark_2)
     {
      case sd0:
         break;
      case sd1:
         first_str=first_str+mark_array[0]+";";
         break;
      case sd2:
         first_str=first_str+mark_array[1]+";";
         break;
      case sd3:
         first_str=first_str+mark_array[2]+";";
         break;
      case sd4:
         first_str=first_str+mark_array[3]+";";
         break;
      case sd5:
         first_str=first_str+mark_array[4]+"="+IntegerToString(ima_day)+";";
         break;
      case sd6:
         first_str=first_str+mark_array[5]+";";
         break;
     }

   switch(Prediction_Mark_3)
     {
      case sd0:
         break;
      case sd1:
         first_str=first_str+mark_array[0]+";";
         break;
      case sd2:
         first_str=first_str+mark_array[1]+";";
         break;
      case sd3:
         first_str=first_str+mark_array[2]+";";
         break;
      case sd4:
         first_str=first_str+mark_array[3]+";";
         break;
      case sd5:
         first_str=first_str+mark_array[4]+"="+IntegerToString(ima_day)+";";
         break;
      case sd6:
         first_str=first_str+mark_array[5]+";";
         break;
     }

   switch(Changepoint_Metod)
     {
      case ch0:
         break;
      case ch1:
         first_str=first_str+"up_fractal;down_fractal;";
         break;
     }
   switch(Regression_Metod)
     {
      case rg0:
         break;
      case rg1:
         first_str=first_str+"regression=oc;";
         break;
      case rg2:
         first_str=first_str+"regression=atr;";
         break;
     }
   switch(Holiday_Metod)
     {
      case hd0:
         break;
      case hd1:
         first_str=first_str+"holidays=country;";
         break;
      case hd2:
         first_str=first_str+"holidays=weekend;";
         break;
     }
   switch(Metod_Processing_Data)
     {
      case md0:
         first_str=first_str+"metod=ideal;";
         break;
      case md1:
         first_str=first_str+"metod=normal;";
         break;
      case md2:
         first_str=first_str+"metod=transform;";
         break;
     }
   first_str=first_str+"pred="+IntegerToString(Pridiction_Period,3,' ');

// Записуємо першу строку датафрейму
   FileWrite(fil,first_str);

// У випадку, якщо сьогодні — торговий день, проведемо корегування кількості барів в історії
   if(Today)
      num_bar=num_bar-1;

// Запускаємо цикл, який порядково буде формувати дані у змінній next_str та записувати цей рядок у файл датафрейму
   for(j=0; j<=num_bar; j++)
     {
      i = bar_shift-j;
      next_str=TimeToString(iTime(name,PERIOD_CURRENT,i),TIME_DATE)+";";
      switch(Prediction_Mark_1)
        {
         case sd0:
            break;
         case sd1:
            next_str=next_str+DoubleToString(iOpen(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd2:
            next_str=next_str+DoubleToString(iHigh(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd3:
            next_str=next_str+DoubleToString(iLow(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd4:
            next_str=next_str+DoubleToString(iClose(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd5:
            next_str=next_str+DoubleToString(ma[j],5)+";";
            break; // 5 - кількість знаків після коми
         case sd6:
            next_str=next_str+DoubleToString(atr[j],5)+";";
            break;
        }

      switch(Prediction_Mark_2)
        {
         case sd0:
            break;
         case sd1:
            next_str=next_str+DoubleToString(iOpen(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd2:
            next_str=next_str+DoubleToString(iHigh(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd3:
            next_str=next_str+DoubleToString(iLow(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd4:
            next_str=next_str+DoubleToString(iClose(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd5:
            next_str=next_str+DoubleToString(ma[j],5)+";";
            break;
         case sd6:
            next_str=next_str+DoubleToString(atr[j],5)+";";
            break;
        }

      switch(Prediction_Mark_3)
        {
         case sd0:
            break;
         case sd1:
            next_str=next_str+DoubleToString(iOpen(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd2:
            next_str=next_str+DoubleToString(iHigh(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd3:
            next_str=next_str+DoubleToString(iLow(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd4:
            next_str=next_str+DoubleToString(iClose(name,PERIOD_CURRENT,i),5)+";";
            break;
         case sd5:
            next_str=next_str+DoubleToString(ma[j],5)+";";
            break;
         case sd6:
            next_str=next_str+DoubleToString(atr[j],5)+";";
            break;
        }

      switch(Changepoint_Metod)
        {
         case ch0:
            break; // виходимо із гіпотези що реально не існує інструменту із ціною більше 10000000$
         case ch1:
           {
            if(up_fractal[j]<10000000 && down_fractal[j]>10000000)
              {
               next_str=next_str+DoubleToString(up_fractal[j],5)+";;";
              }
            if(up_fractal[j]>10000000 && down_fractal[j]<10000000)
              {
               next_str=next_str+";"+DoubleToString(down_fractal[j],5)+";";
              }
            if(up_fractal[j]<10000000 && down_fractal[j]<10000000)
              {
               next_str=next_str+DoubleToString(up_fractal[j],5)+";"+DoubleToString(down_fractal[j],5)+";";
              }
            if(up_fractal[j]>10000000 && down_fractal[j]>10000000)
              {
               next_str=next_str+";;";
              }
            break;
           }
        }
      switch(Regression_Metod)
        {
         case rg0:
            break;
         case rg1:
            next_str=next_str+DoubleToString(iOpen(name,PERIOD_CURRENT,i)-iClose(name,PERIOD_CURRENT,i),5)+";";
            break;
         case rg2:
            next_str=next_str+DoubleToString(atr[j],5)+";";
            break;
        }
      switch(Holiday_Metod)
        {
         case hd0:
            break;
         case hd1:
            next_str=next_str+";";
            break;
         case hd2:
            next_str=next_str+";";
            break;
        }

      FileWrite(fil,next_str);     // Записуємо строку у файл
     }

// Закриваємо сформований файл
   FileClose(fil);

// Виводимо в журнал радника повідомлення про те, що файл із даними сформовано
   Print("The program has generated a data file ",filename);
   Print("Now you need to run the script Prophet_model.");

// З метою уникнення конфліктів, якщо Prediction_Mark=atr та regression=atr, проведемо коригування строки, яка формувала заголовок датафрейму
// Аналогічно для normal та ma
   StringReplace(first_str,"regression=atr","regression");
   StringReplace(first_str,"normal","norm");

//  Формуємо назви файлів, у яких будуть перевірятися передбачення. Для цього створюємо масив mk, у якому зберігаються реальні стовбчики датафрейму
   num_mk=0;  // Кількість елементів масиву mk
   for(i=0; i<6; i++)
     {
      if(StringFind(first_str,mark_array[i])>0)
        {
         mk[num_mk]=mark_array[i];
         num_mk++;
        }
     };

// Перетворимо дату End у структуру для отримання компонентів дати
   TimeToStruct(End,ennd);

// Перетворюємо масив із назвами стовбчиків, за якими було проведено передбачення, у массив із назвами відповідних файлів, які сформував Prophet_model
   for(i=0; i<num_mk; i++)
     {
      mk[i]="Forecast"+"─"+mk[i]+"─"+name+"─"+StringFormat("%04d",ennd.year)+"-"+StringFormat("%02d",ennd.mon)+"-"+StringFormat("%02d",ennd.day)+".csv";
     }

// Зупиняємо процес виконання програми до того моменту, поки  Prophet_model не сформує файли прогнозів для їх подальшої відмальовки
// Перевіряємо, чи всі файли створені, якщо ні – чекаємо 1000 мілісекунд та повторюємо процес
   while(true)
     {
      j=0;
      for(i=0; i<num_mk; i++)
        {
         if(FileIsExist(mk[i],0))
            j++;
        }
      Sleep(1000);
      if(j==num_mk)
         break;
     };

//  Формуємо сповіщення, що файли передбачення створені
   Print("Prediction files are ready. Now create charts.");

// Починаємо відмальовувати графіки за кожним Prediction_Mark
   for(i=0; i<num_mk; i++)  
     {
      // Відкриваємо відповідний файл
      fil=FileOpen(mk[i],FILE_READ|FILE_CSV|FILE_ANSI);

      // Зчитуємо перший рядок файлу, в якому міститься інформація про назву Prediction_Mark та МАЕ
      rab_str=FileReadString(fil);

      // Розділяємо рядок на масив
      StringSplit(rab_str,StringGetCharacter(",",0),str_result);

      // Змінна назви графіку
      name_graph[i]=str_result[2];

      // Завантажуємо дані з файлу та приводимо їх до відповідного формату, закриваємо файл
      k=0;
      while(!FileIsEnding(fil))
        {
         rab_str=FileReadString(fil);
         StringSplit(rab_str,StringGetCharacter(",",0),str_result);
         load_time[k]=StringToTime(str_result[1]);
         load_data[i][0][k]=StringToDouble(str_result[2]);
         load_data[i][1][k]=StringToDouble(str_result[3]);
         load_data[i][2][k]=StringToDouble(str_result[4]);
         k++;
        }
      FileClose(fil);

      // В змінній k знаходиться кількість точок для малювання
      // Проведемо корекцію дат у відповідності до прогнозу, якщо прогноз випадає на суботу чи неділю, просто зміщуємо прогнозні дані на 2 дні

      for(m=k-Pridiction_Period-1; m<k; m++)
        {
         TimeToStruct(load_time[m],ennd);
         if(ennd.day_of_week == 5)
           {
            for(l=m+1; l<k; l++)
              {
               TimeToStruct(load_time[l],bar_time);
               bar_time.day+=2;
               load_time[l]=StructToTime(bar_time);
              }
           }
        }

     }
// Встановлюємо час реакції таймера 1с.
   EventSetTimer(1);

   return(INIT_SUCCEEDED);
  }

//+------------------------------------------------------------------+
//|                                                                  |
//+------------------------------------------------------------------+
void OnTimer()
  {
// Визначаємо номер бара, який потрапляє на точку End
   bar_shift=iBarShift(Symbol(),PERIOD_CURRENT,End);

// Створюємо ім'я для графічного об'єкту МАЕ
   obj_name_ma="MAE:\n";

// Починаємо створювати графічні об'єкти по кожній компоненті
   for(i=0; i<num_mk; i++)   // i<num_mk; поменять
     {
      // За необхідності обраховуємо значення МАЕ
      if(Calc_MAE)
        {
         //Отримаємо значення MA на період предикції
         if(name_graph[i]=="ma")
           {
            if(CopyBuffer(IMA1,0,End,load_time[k-1],ma) < 0)
              {
               Print("CopyBuffer IMA1 error =",GetLastError());
              }
           }
         MAE=0;
         l=0;
         // Проводимо обрахунок МАЕ за кожним параметром окремо
         for(m=k-Pridiction_Period; m<k; m++)
           {
            if(name_graph[i]=="open")
               MAE=MAE+MathAbs(iOpen(name,PERIOD_CURRENT,bar_shift-l-1)-load_data[i][0][m]);
            if(name_graph[i]=="high")
               MAE=MAE+MathAbs(iHigh(name,PERIOD_CURRENT,bar_shift-l-1)-load_data[i][0][m]);
            if(name_graph[i]=="low")
               MAE=MAE+MathAbs(iLow(name,PERIOD_CURRENT,bar_shift-l-1)-load_data[i][0][m]);
            if(name_graph[i]=="close")
               MAE=MAE+MathAbs(iClose(name,PERIOD_CURRENT,bar_shift-l-1)-load_data[i][0][m]);
            if(name_graph[i]=="ma")
               MAE=MAE+MathAbs(ma[l+1]-load_data[i][0][m]);
            if(name_graph[i]=="atr")
               MAE=MAE+MathAbs((iHigh(name,PERIOD_CURRENT,bar_shift-l-1)-iLow(name,PERIOD_CURRENT,bar_shift-l-1))-load_data[i][0][m]);
            l++;
           }
         // Перераховуємо значення МАЕ у пункти та вираховуємо середнє значення
         MAE=MAE*pow(10,Digits())/Pridiction_Period;

         // Готуємо ім'я графічного об'єкту МАЕ для виводу
         obj_name_ma+=name_graph[i]+" : "+DoubleToString(MAE,0)+" \n";
        }

      // Створюємо графік в основному вікні терміналу
      for(m=0; m<k-1; m++)
        {
         if(name_graph[i]=="atr")
           {
            // Запускаємо нове вікно індикатора ATR
            ChartIndicatorAdd(0,1,IATR1);
            // Створюємо ім'я графічного об'єкту
            obj_name="ATR-"+TimeToString(load_time[m],TIME_DATE)+" "+DoubleToString(load_data[i][0][m],5)+"  ↑"+DoubleToString(load_data[i][2][m],5)+" ↓"+ DoubleToString(load_data[i][1][m],5);
            // Малюємо об'єкт
            if(!ObjectCreate(0,obj_name,OBJ_TREND,1,load_time[m],load_data[i][0][m],load_time[m+1],load_data[i][0][m+1]))
              {
               Print("Error creation object : ", obj_name, " code #",GetLastError());
              }
           }
         else
           {
            // Створюємо ім'я графічного об'єкту основного графіку
            obj_name=name_graph[i]+"-"+TimeToString(load_time[m],TIME_DATE)+" "+DoubleToString(load_data[i][0][m],5)+"  ↑"+DoubleToString(load_data[i][2][m],5)+" ↓"+ DoubleToString(load_data[i][1][m],5);
            // Малюємо об'єкт
            if(!ObjectCreate(0,obj_name,OBJ_TREND,0,load_time[m],load_data[i][0][m],load_time[m+1],load_data[i][0][m+1]))
              {
               Print("Error creation object : ", obj_name, " code #",GetLastError());
              }
           }
         // Змінюємо колір у періоді передбачення
         if(m<k-Pridiction_Period-1)
            ObjectSetInteger(0, obj_name, OBJPROP_COLOR, clrFloralWhite);
         else
            ObjectSetInteger(0, obj_name, OBJPROP_COLOR, clrMagenta);
        }

      // Виводимо графіки на екран
      ChartRedraw();
     }
// Якщо потрібно, виводимо графік на екран
   if(Calc_MAE)
     {
      // Розраховуємо точку виводу графічного об'єкту МАЕ
      XMAE = iHigh(name,PERIOD_CURRENT,0);

      // Створюємо графічний об'єкт
      ObjectCreate(0, obj_name_ma, OBJ_TEXT,0,load_time[k-1],XMAE);

      // Встановлюємо його властивості
      ObjectSetInteger(0, obj_name_ma, OBJPROP_ANCHOR, ANCHOR_LEFT_UPPER);
      ObjectSetString(0, obj_name_ma, OBJPROP_TEXT, "MAE");
      ObjectSetString(0, obj_name_ma, OBJPROP_FONT, "Arial");
      ObjectSetInteger(0, obj_name_ma, OBJPROP_FONTSIZE,15);
      ObjectSetInteger(0, obj_name_ma, OBJPROP_COLOR, clrRed);
      ObjectSetInteger(0, obj_name_ma, OBJPROP_SELECTABLE, true);

      // Виводимо об'єкт на екран
      ChartRedraw();
     }
// Змінюємо час таймеру для того, щоб графіки не перебудовувалися раніше ніж 5 хвилин
   EventSetTimer(300);
  }

//+------------------------------------------------------------------+
void OnTick()
  {
//---
  }

//+------------------------------------------------------------------+
void OnDeinit(const int reason)
  {
// Прибираємо всі графіки з екрану
   ObjectsDeleteAll(0,0,-1);
   if(!FileDelete(filename))
     {
      Print("Error of deleting file ",filename);
     }
   for(i=0; i<num_mk; i++)
     {
      if(name_graph[i]=="atr")
        {
         ObjectsDeleteAll(0,1,-1);
        }
      if(!FileDelete(mk[i]))
        {
         Print("Error of deleting file ",mk[i]);
        }
     }
// Ліквідуємо таймер
   EventKillTimer();
   Print("Prophet_Start process completed successfull!");
  }
//+------------------------------------------------------------------+


#### Код програми Prophet_Model

In [ ]:
# Програма Prophet_model написана на базі моделі лінійної регресії Prophet (https://facebook.github.io/prophet/) для
# прогнозування курсів фінансових інструментів та працює в парі з торговим радником Prophet_Start на торговому терміналі
# МТ5 в режимі скрипта Python. Для її роботи необхідно установити програму в каталог скриптів торгового терміналу МТ5 та
# провести її запуск у середовищі MetaEditor, яке є невід`ємною частиною торгового терміналу МТ5. Після цього програма
# з`явиться у каталозі скриптів торгового терміналу з поміткою скрипт Python.
# Програма працює виключно на таймфреймі D1.
# За основу алгоритма прийнято три гіпотези для побудови прогнозів:
# 1. Модель ideal - коли реальні торгові дані перетворюються в ідеальний календар, який не містить пропусків даних, що
#    обумовлено вихідними та святковими днями, під час яких не проходять торги.
# 2. Модель normal - коли дані для моделі готуються з пустими значеннями, які припадають на вихідні та свята. При цьому
#    можуть враховуватися вихідні дні з розподіленим впливом на п`ятницю та понеділок методом holidays (window), описаним
#    в технічній документації моделі Prophet.
# 3. Модель Transform - яка проводить корегування даних та заповнює пусті значення плавно розподіленими середніми
#    значеннями між точками до свята та після свята.
# В якості додаткових регрессорів всі моделі можуть використовувати:
#   - вихідні дні офіційних свят, прийнятих для NYSE, або модель, коли всі незаповнені дні вважаються вихідними;
#   - використовувати фрактали Вільямса як точки changepoints згідно з документацією моделі Prophet;
#   - брати за основу два регресійних фактори Open-Close (ос) або ATR (atr) та додавати їх до обрахунків.
#  В результаті роботи програма створює файли прогнозів згідно зі стандартами обміну файлами між Prophet_model та Prophet_Start
#  Вхідний файл для програми - це останній (за часом)csv файл, створений у спеціальному середовищі терміналу MT5 в каталозі Files
#  В якості вихідних файлів програма формує кількість файлів у відповідності до кількості прогнозованих показників.
#  обраних у модулі Prophet_Start. Стандарт створення файлів Forecast─high─GBPCAD─2023-04-07.csv.
#  Де high - параметр предикції, може приймати значення (open,high,low,close,ma,atr).
#  GBPCAD - фінансовий інструмент, на якому буде проводитися предикція. Будь-який із доступних в терміналі.
#  2023-04-07 - дата у форматі YYYY-MM-DD, яка відповідає останній даті періоду навчання моделі.
#  З цієї дати буде проводиться предикція.
#
#
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from prophet import Prophet
import os
import holidays
import matplotlib.pyplot as plt
from prophet.plot import add_changepoints_to_plot
import MetaTrader5 as mt5
import warnings
warnings.filterwarnings("ignore")

# Процедура готує дані для моделі Ideal
def ideal(df1, ideal_mark, predict_num,  holiday_method, regression_method, changepoint_method):
    hol_day_method = None    # метод ideal не дозволяє використовувати свята в якості предиктора
    num_date = len(df1)
    ser_regress = pd.Series()    # Змінна, в якій буде створюватися додатковий регресійний фактор
    result = pd.DataFrame(columns=['ds', 'y'], index=pd.RangeIndex(0, num_date))    # результуючий датафрейм
# Цикл змінює реальні дати на ідеальний календар без вихідних та свят
    for ii in range(num_date):
        result['ds'][num_date - 1 - ii] = (df1['ds'][num_date - 1] - timedelta(days=ii))
# Формуємо дані та розраховуємо додатковий регресійний фактор
    if regression_method == "oc":
        result['y'] = df1['regression=oc'][:].values
        ser_regress = model_reg(result, predict_num)
    if regression_method == "atr":
        result['y'] = df1['regression=atr'][:].values
        ser_regress = model_reg(result, predict_num)
#  За необхідності формуємо changepoint_list з фракталів
    if changepoint_method == "fractals":
        result['up_fractal'] = df1['up_fractal'][:].values
        result['down_fractal'] = df1['down_fractal'][:].values
        cng_lst=changepoint(result)
    else:
        cng_lst = None
#  Формуємо стовбчик 'y' для предиції  згідно з документацією моделі  Prophet
    result['y'] = df1[ideal_mark][:].values
    if 'up_fractal' in result.columns:
        result.drop('up_fractal', axis=1, inplace=True)
    if 'down_fractal' in result.columns:
        result.drop('down_fractal', axis=1, inplace=True)
    return result, hol_day_method, ser_regress, cng_lst


# Процедура готує дані для моделі Normal
def normal(df1, normal_mark, predict_num,  holiday_method, regression_method, changepoint_method):
    result = pd.DataFrame(columns=['ds', 'y'])
    result['ds'] = df1['ds']
    ser_regress = pd.Series()
# Формуємо дані та розраховуємо додатковий регресійний фактор
    if regression_method == "oc":
        result['y'] = df1['regression=oc'][:].values
        ser_regress = model_reg(result, predict_num)
    if regression_method == "atr":
        result['y'] = df1['regression=atr'][:].values
        ser_regress = model_reg(result, predict_num)
#  За необхідності формуємо changepoint_list з фракталів
    if changepoint_method == "fractals":
        cng_lst = changepoint(df1)
    else:
        cng_lst = None
#  Формуємо стовбчик 'y' для предиції  згідно документації моделі  Prophet
    result['y'] = df1[normal_mark][:].values
#  За необхідності формуємо фактор свят
    if holiday_method == "country":
        hol_day_method = get_holiday_country(result, predict_num)
    elif holiday_method == "weekend":
        hol_day_method = get_holiday_weekend(result, predict_num, True)
    else:
        hol_day_method = None
    return result, hol_day_method, ser_regress, cng_lst


# Процедура трансформує данні та заповнює пробіли середніми значеннями для моделі Transform
def transform_mark(df1, trans_mk):
    start_tr = df1['ds'][0]           # Дата початку обрахунків
    end_tr = df1['ds'][len(df) - 1]   # Дата кінця обрахунків
    result = pd.DataFrame(columns=['ds', 'y'], index=pd.RangeIndex((end - start).days + 1))
# Генеруємо повний календар дат
    result['ds'] = pd.date_range(start_tr, end_tr, freq='d')
    jj = 0
    kk = 0
    ii = 0
    while ii < len(result):
        if result['ds'][ii] == df1['ds'][jj]:        # Заповнюємо існуючі дати
            result['y'][ii] = df1[trans_mk][jj]
            jj += 1
            ii += 1
        else:       # Заповнюємо не існуючі дати
            kk = (df1['ds'][jj] - df1['ds'][jj - 1]).days - 1  # Кількість вихідних днів
            delta = (df1[trans_mk][jj] - df1[trans_mk][jj - 1]) / (kk + 1)  # Денний приріст ціни у вихідний день
            new = df1[trans_mk][jj - 1]
            for mm in range(0, kk):
                result['y'][ii + mm] = new + delta * (mm + 1)
            ii = ii + kk
    return result


# Процедура готує дані безпосередньо для моделі Transform
def transform(df1, trans_mk, predict_num,  holiday_method, regression_method, changepoint_method):
    ser_regress = pd.Series()
    result = pd.DataFrame()
# Формуємо дані та розраховуємо додатковий регресійний фактор
    if regression_method == "oc":
        result = transform_mark(df1, 'regression=oc')
        ser_regress = model_reg(result, predict_num)
    if regression_method == "atr":
        result = transform_mark(df1, 'regression=atr')
        ser_regress = model_reg(result, predict_num)
#  За необхідности формуємо changepoint_list з фракталів
    if changepoint_method == "fractals":
        cng_lst = changepoint(df1)
    else:
        cng_lst = None
#  Формуємо стовбчик 'y' для предикції  згідно з документацією моделі Prophet
    result = transform_mark(df1, trans_mk)
#  За необхідності формуємо фактор свят
    if holiday_method == "country":
        hol_day_method = get_holiday_country(result, predict_num)
    elif holiday_method == "weekend":
        hol_day_method = get_holiday_weekend(result, predict_num)
    else:
        hol_day_method = None
    return result, hol_day_method, ser_regress, cng_lst


# Процедура готує дані про свята для моделі Transform, Normal
def get_holiday_weekend(df1, predict_n, window=False):
    start_hol = df1['ds'][0]                # Дата початку обрахунків
    end_hol = df1['ds'][len(df1) - 1]       # Дата кінця обрахунків
    end_predict = df1['ds'][len(df1) - 1] + pd.Timedelta(days=predict_n)   # Дата кінця предикції
# Створюємо новий повний календар
    predict_lst = pd.date_range(end_hol, end_predict, freq='d')
    df_hol_day = pd.DataFrame(columns=['ds', 'holiday'])
    df_hol_day['ds'] = pd.date_range(start_hol, end_hol, freq='d')
# Визначаємо співпадіння індексів
    idx = df_hol_day[df_hol_day['ds'].isin(df1['ds'])].index
    df_hol_day.drop(idx, inplace=True)
    df_hol_day.reset_index(drop=True, inplace=True)
    end_index = len(df_hol_day)
# Отримуємо святкові дні за розкладом NYSE
    nyse_holiday = holidays.NYSE(years=[end_hol.year, end_predict.year], observed=True)
# Перевіряємо співпадіння дат зі святами та заповнюємо значення згідно з вимогами моделі Prophet
    for ii in range(1, predict_n + 1):
        if predict_lst[ii].day_of_week == 5 or predict_lst[ii].day_of_week == 6 or predict_lst[ii] in nyse_holiday:
            df_hol_day.loc[end_index] = [predict_lst[ii], 'weekend']
            end_index += 1
    df_hol_day['holiday'] = 'weekend'
# Заповнюємо значення  lower_window та upper_window згідно з вимогами моделі Prophet
    if window:
        df_hol_day['lower_window'] = -1
        df_hol_day['upper_window'] = 1
    return df_hol_day

# Процедура готує дані про робочі дні NYSE для моделі Transform, Normal
def get_holiday_country(df1, predict_n):
    start_hol=df1['ds'][0]
    end_hol=df1['ds'][len(df1)-1]
    year_list = [ii for ii in range(start_hol.year, end_hol.year + 1, 1)]
    nyse_holidays = holidays.NYSE(years=year_list, observed=True)
    df_hol_day = pd.DataFrame(nyse_holidays.items(), columns=['ds', 'holiday'])
    df_hol_day['ds'] = pd.to_datetime(df_hol_day['ds'])
    return df_hol_day

# Процедура здійснює пошук та ліквідацію хибних фракталів, а також готує changepoint list для роботи моделі
def changepoint(df1):
    change_lst = []
    jj = 0
    ii = 0
    while ii <= len(df1) - 1 or jj <= len(df1) - 1:  # цикл проходу по історії фракталів
# Додаємо у лист верхні не хибні фрактали
        if df1['up_fractal'][ii] > 0 and not df1['down_fractal'][ii] > 0:
            temp_lst_up = []  # створюємо тимчасовий лист для збереження up фракталів
            jj = ii
            temp_lst_up.append([ii, df1['up_fractal'][ii]])
            while not df1['down_fractal'][jj] > 0:
                jj += 1
                if jj >= len(df1):
                    break
                if df1['up_fractal'][jj] > 0:
                    temp_lst_up.append([jj, df1['up_fractal'][jj]])
            if jj >= len(df1):
                change_lst.append(df1['ds'][ii])
                break
            if df1['up_fractal'][jj] > 0:
                temp_lst_up.append([jj, df1['up_fractal'][jj]])

            if len(temp_lst_up) == 1:
                change_lst.append(df1['ds'][ii])
                ii += 1
                continue
            else:
                max_up = temp_lst_up[0][1]
                indx = temp_lst_up[0][0]
                for mm in temp_lst_up:
                    if mm[1] > max_up:
                        max_up = mm[1]
                        indx = mm[0]
                for mm in temp_lst_up:
                    if mm[0] != indx:
                        df1.loc[mm[0], 'up_fractal'] = np.NaN

                    else:
                        change_lst.append(df1['ds'][indx])
            ii = jj
            continue
# Додаємо у лист нижні не хибні фрактали
        if df1['down_fractal'][ii] > 0 and not df1['up_fractal'][ii] > 0:
            temp_lst_down = []   # створюємо тимчасовий лист для збереження down фракталів
            jj = ii
            temp_lst_down.append([ii, df1['down_fractal'][ii]])
            while not df1['up_fractal'][jj] > 0:
                jj += 1
                if jj >= len(df1):
                    break
                if df1['down_fractal'][jj] > 0:
                    temp_lst_down.append([jj, df1['down_fractal'][jj]])
            if jj >= len(df1):
                change_lst.append(df1['ds'][ii])
                break
            if df1['down_fractal'][jj] > 0:
                temp_lst_down.append([jj, df1['down_fractal'][jj]])

            if len(temp_lst_down) == 1:
                change_lst.append(df1['ds'][ii])
                ii += 1
                continue
            else:
                min_dn = temp_lst_down[0][1]
                indx = temp_lst_down[0][0]
                for mm in temp_lst_down:
                    if mm[1] < min_dn:
                        min_dn = mm[1]
                        indx = mm[0]
                for mm in temp_lst_down:
                    if mm[0] != indx:
                        df1.loc[mm[0], 'down_fractal'] = np.NaN
                    else:
                        change_lst.append(df1['ds'][indx])
            ii = jj
            continue
#  Заповнюємо датафрейм значеннями фракталів
        if df1['down_fractal'][ii] > 0 and df1['up_fractal'][ii] > 0:
            jj = ii
            while not (df1['down_fractal'][jj + 1] > 0) and not (df1['up_fractal'][jj + 1] > 0):
                jj += 1
                if jj >= len(df1):
                    break
            jj += 1
            if jj >= len(df1):
                change_lst.append(df1['ds'][ii])
                break
            if df1['down_fractal'][jj] > 0 and not df1['up_fractal'][jj] > 0:
                df1.loc[ii, 'down_fractal'] = np.NaN

            if df1['up_fractal'][jj] > 0 and not df1['down_fractal'][jj] > 0:
                df1.loc[ii, 'up_fractal'] = np.NaN

            if df1['down_fractal'][jj] > 0 and df1['up_fractal'][jj] > 0:

                if df1['down_fractal'][ii] < df1['down_fractal'][jj] and df1['up_fractal'][ii] > df1['up_fractal'][jj]:
                    df1.loc[jj, 'down_fractal'] = np.NaN
                    df1.loc[jj, 'up_fractal'] = np.NaN

                if df1['down_fractal'][ii] > df1['down_fractal'][jj] and df1['up_fractal'][ii] < df1['up_fractal'][jj]:
                    df1.loc[ii, 'down_fractal'] = np.NaN
                    df1.loc[ii, 'up_fractal'] = np.NaN

                if df1['down_fractal'][ii] < df1['down_fractal'][jj] and df1['up_fractal'][ii] < df1['up_fractal'][jj]:
                    df1.loc[ii, 'up_fractal'] = np.NaN

                if df1['down_fractal'][ii] > df1['down_fractal'][jj] and df1['up_fractal'][ii] > df1['up_fractal'][jj]:
                    df1.loc[ii, 'down_fractal'] = np.NaN

            continue
        ii += 1
# Приводимо список до вимог моделі Prophet
    change_lst = set(change_lst)
    change_lst = list(change_lst)
    change_lst.sort()
    return change_lst

# Функція видаляє вихідні дні у предикшен періоді для моделі normal
def result_processing(df1, predict_n):
    nn=len(df1)-1
    for ii in range(predict_n):
        if df1['ds'][nn-ii].weekday() == 5 or df1['ds'][nn-ii].weekday() == 6:
            df1=df1.drop(nn-ii)
    df1.reset_index(drop=True, inplace=True)
    return df1

# Функція будує регрессійний фактор на базі моделі
def model_reg(df_reg, predict_num):
    m = Prophet(holidays=None,
               holidays_prior_scale=10,
               changepoint_range=0.95,
               changepoint_prior_scale=0.1,
               yearly_seasonality='auto',
               weekly_seasonality='auto',
               daily_seasonality=0,
               growth='linear',
               n_changepoints=100,
               seasonality_mode='additive',
               seasonality_prior_scale=10,
               mcmc_samples=0,
               interval_width=0.1,
               uncertainty_samples=1000,
               stan_backend=None)

    m.fit(df_reg)
    m_future = m.make_future_dataframe(periods=predict_num)
    m_forecast = m.predict(m_future)
    return m_forecast['yhat']



# Ініціалізуємо бібліотеку mt5
mt5.initialize()

# Знаходимо останній змінений файл
path = 'C:\\Users\\Александр\\AppData\\Roaming\\MetaQuotes\\Terminal\\550A9E6B699B2474CBE17711F34DD758\\MQL5\\Files\\'
files = os.listdir(path)
max_date = os.path.getmtime(path + files[0])
n = 0
for i in range(len(files)):
    if os.path.getmtime(path + files[i]) > max_date:
        max_date = os.path.getmtime(path + files[i])
        n = i
# Отримуємо ім'я фінансового інструмента
ticker = files[n][4:10]

# Читаємо файл
df = pd.read_csv(path + files[n], delimiter=';')

# Перетворюємо дати у формат datetime
df['ds'] = pd.to_datetime(df['ds'])

# Визначаємо точку старту обрахунків
start = df['ds'][0]

# Визначаємо точку кінця обрахунків
end = df['ds'][len(df) - 1]

print("Prophet_model start calculation on file ", files[n], "  Instrument : ", ticker, "  Period : ", start.strftime("%Y-%m-%d"), "-", end.strftime("%Y-%m-%d"))

# Із заголовка файлу отримуємо інформацію стосовно методів та параметрів моделі
columns = df.columns
mark = []
holiday_method = "None"
regression_method = "None"
changepoint_method = "None"
predict_num = int(columns[-1].split("=")[1])
method = columns[-2].split("=")[1]
for i in columns:
    if len(i.split("=")) > 1:
        if i.split("=")[0] == "holidays":
            holiday_method = i.split("=")[1]
        if i.split("=")[0] == "regression":
            regression_method = i.split("=")[1]
        if i.split("=")[0] == "changepoint":
            changepoint_method = i.split("=")[1]
        if i.split("=")[0] == "ma":
            ma_num = int(i.split("=")[1])
            mark.append(i.split("=")[0])
            df = df.rename(columns={i: 'ma'})
    if i == "up_fractal":
        changepoint_method = "fractals"
    if i == 'open':
        mark.append(i)
    if i == 'high':
        mark.append(i)
    if i == 'low':
        mark.append(i)
    if i == 'close':
        mark.append(i)
    if i == 'atr':
        mark.append(i)

# Формуємо необхідні параметри моделі
hol_prior_scale=10
n_changepoint=25
chang_range=0.95
chang_prior_scale=0.05
year_seasonality='auto'
week_seasonality='auto'
day_seasonality=0
growth_ln='linear'
season_mode='additive'
season_prior_scale=10
mcmc=0
interval=0.05
samples=1000
stan=None
hol_day_method = None
changepoint_lst = None

# Відкриваємо файл журналу
file_jr = open('C:\\Users\\Александр\\AppData\\Roaming\\MetaQuotes\\Terminal\\550A9E6B699B2474CBE17711F34DD758\\MQL5\\Files\\prophet_journal.csv', 'a')

# Починаємо розрахунки за кожною змінною та результати кожного передбачення зберігаємо у файл 
df_rab = pd.DataFrame()
series_regress = pd.Series()

for i in mark:
# Відпрацьовуємо метод Ideal
    if method == "ideal":
        df_rab, hol_day_method, series_regress, changepoint_lst = ideal(df, i, predict_num,  holiday_method, regression_method, changepoint_method)
        if regression_method == "oc" or regression_method == "atr":
            df_rab['regress'] = series_regress
# Відпрацьовуэмо метод Normal
    if method == "normal":
        df_rab, hol_day_method, series_regress, changepoint_lst = normal(df, i, predict_num,  holiday_method, regression_method, changepoint_method)
        if regression_method == "oc" or regression_method == "atr":
            df_rab['regress'] = series_regress
# Відпрацьовуэмо метод Transform
    if method == "transform":
        df_rab, hol_day_method, series_regress, changepoint_lst = transform(df, i, predict_num,  holiday_method, regression_method, changepoint_method)
        if regression_method == "oc" or regression_method == "atr":
            df_rab['regress'] = series_regress
         
# Запускаємо модель на розрахунок
    model = Prophet(holidays=hol_day_method,
                    holidays_prior_scale=hol_prior_scale,
                    changepoints=changepoint_lst,
                    n_changepoints=n_changepoint,
                    changepoint_range=chang_range,
                    changepoint_prior_scale=chang_prior_scale,
                    yearly_seasonality=year_seasonality,
                    weekly_seasonality=week_seasonality,
                    daily_seasonality=day_seasonality,
                    growth=growth_ln,
                    seasonality_mode=season_mode,
                    seasonality_prior_scale=season_prior_scale,
                    mcmc_samples=mcmc,
                    interval_width=interval,
                    uncertainty_samples=samples,
                    stan_backend=stan)

# За необхідності додаємо до моделі регресійний фактор
    if regression_method == "oc" or regression_method == "atr":
        model.add_regressor('regress')
# Навчання моделі
    model.fit(df_rab)
# Підготовка даних майбутніх періодів
    future_new = model.make_future_dataframe(periods=predict_num)
# При застосуванні регресійних факторів додаємо регресійні фактори майбутніх періодів
    if regression_method == "oc" or regression_method == "atr":
        future_new['regress'] = series_regress
# Робимо передбачення
    forecast_new = model.predict(future_new)
    
# Зберігаємо файл з даними передбачення
    file_forecast = "Forecast─" + i + "─" + ticker + "─" + end.strftime("%Y─%m─%d")+".csv"
    forecast_new.rename(columns={'yhat': i}, inplace=True)
    forecast_new[['ds', i, 'yhat_lower', 'yhat_upper']].to_csv('C:\\Users\\Александр\\AppData\\Roaming\\MetaQuotes\\Terminal\\550A9E6B699B2474CBE17711F34DD758\\MQL5\\Files\\' + file_forecast)   

# Зберігаємо дані роботи в журнал. Для цього необхідно обрахувати МАЕ
# Перевіряємо, чи є необхідність в обрахунку МАЕ 
    mae = None
    if end+timedelta(days=predict_num+1) < datetime.today():
     # Визначаємо періоди обрахунку МАЕ   
        first_day_predict = end+timedelta(days=1)
        end_day_predict = end+timedelta(days=predict_num)
        calc_day=first_day_predict
        num_weekend=0
     # Додаємо вихідні дні у період розрахунку
        while calc_day < end+timedelta(days=predict_num+1):
            if calc_day.weekday() == 5:
                num_weekend +=2
            calc_day = calc_day + timedelta(days=1)
        end_day_predict += timedelta(days=num_weekend)
        
     # Завантажуємо дані з терміналу
        mae_period = pd.DataFrame(mt5.copy_rates_from(ticker, mt5.TIMEFRAME_D1, end_day_predict, predict_num))
        mae_period['time']=pd.to_datetime(mae_period['time'], unit='s')
        mae_period['atr']=mae_period['high']-mae_period['low']
        
     # Вираховуємо МАЕ  
        if i != "ma":
            mae=0
            for q in range(predict_num):
                mae += abs(mae_period[i][q] - forecast_new[i][len(forecast_new)-predict_num+q])
            mae=mae/predict_num                
                
# Готуємо дані для запису в журнал
    journal_str=str(mae)+";"+start.strftime('%Y-%m-%d')+";"+end.strftime('%Y-%m-%d')+";"+i+";"+method+";"+str(predict_num)+";"
    journal_str+=str(changepoint_method)+";"+str(holiday_method)+";"+str(regression_method)+";"+str(chang_range)+";"
    journal_str+=str(chang_prior_scale)+";"+str(hol_prior_scale)+";"+str(year_seasonality)+";"+str(week_seasonality)+";"
    journal_str+=str(day_seasonality)+";"+str(growth_ln)+";"+str(n_changepoint)+";"+str(season_mode)+";"
    journal_str+=str(season_prior_scale)+";"+str(mcmc)+";"+str(interval)+";"+str(samples)+";"+str(stan)+"\n"
   
# Записуємо строку в журнал
    file_jr.write(journal_str)
    
# Завершення роботи скрипта
# Закриваємо журнал
file_jr.close()
# Розриваємо зв`язок із терміналом
mt5.shutdown()
print("Prophet_model completed successfully!")  
    
# Цей розділ закоментованого коду є виключно технічним та слугує для вивчення поведінки моделі
# Корекція методу  normal – це службовий модуль, викликаний тим, що модель Prophet видає неадекватно великий викид даних
# при предикції моделі normal у вихідні дні, особливо на короткому проміжку навчання.
            # Для методу normal викидаємо з виборки передбачення вихідні дні
            #    print(forecast_new)
            #    if method == "normal":
            #        forecast_new = result_processing(forecast_new, predict_num)
            #        print(forecast_new)
               # Залишаємо технічну можливість при необхідності візуалізувати вплив гіперпараметрів моделі   
            #    fig = model.plot(forecast_new) 
            #    from prophet.plot import add_changepoints_to_plot
            #    a = add_changepoints_to_plot(fig.gca(), model_regres, forecast_new)       
            #    fig.show()
            #    plt.show()
                    
            # Зберігаємо повний файл для аналізу з даними передбачення
            #    file_forecast = "Forecast─" + i + "─" + ticker + "─" + end.strftime("%Y─%m─%d") + "-full.csv"
            #    forecast_new.to_csv('C:\\Users\\Александр\\AppData\\Roaming\\MetaQuotes\\Terminal\\550A9E6B699B2474CBE17711F34DD758\\MQL5\\Files\\' + file_forecast)
   
